## Specific conductance, dissolved load, and calcite saturation state

Specific conductance (SpC) is an indicator of the dissolved load in the water. At our site, we expect most of that dissolved load to be from carbonate minerals. However, it could also indicate polution. In that case, other ions, such as Chloride (which we measured) could also be high.

#### Questions

- How does dissolved load change from one part of the system to another?
- Is there any evidence that these differences are a function of polution, or could it just be differences in dissolved carbonate?
- Is the water undersaturated, saturated, or supersaturated with respect to calcite?
- Does evolution of SpC along the sinking stream show any evidence of precipitation or dissolution, or can the changes in SpC be explained purely by mixing of waters from the springs and stream?

#### Read in water quality data

In [3]:
# Read in the water quality data from the spreadsheet



In [ ]:
# Make a boxplot of SpC measurements categorized by type of site
# (spring, epikarst, stream, well)



### SpC and pollution

In [ ]:
# To examine whether there is any relationship between SpC and
# pollution, make a scatter plot of SpC vs Chloride.



### Calcite Saturation

To examine calcite saturation, we will use functions from a Python package called *Olm*, which was written to conduct basic water chemistry analysis in karst settings. First we need to install this package (if you haven't already done so). We can install using the following command.

In [1]:
!pip install olm-karst

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 701.6 kB/s eta 0:00:00 kB/s eta 0:00:01:01


To see more about what *Olm* can do, check out the Github repository here:

https://github.com/CovingtonResearchGroup/olm/tree/master

Or the example tutorials here:

https://github.com/CovingtonResearchGroup/olm-examples

We will use *Olm* here to estimate the saturation state of the water with respect to calcite (the main component of limestone). These calcualtions assume that the chemistry is dominated by ions in the $H_2O - CO_2 - CaCO_3$ system. If we have a substantial component from other ions (for example from pollution), then these calculations will be inaccurate.

The first step is to estimate the dissolved Calcium concentration from our SpC values. This could be done using a global relationship which is provided in olm.general.CaFromCond(). However, we can do better, since we have a set of Ca concentrations and SpC measurements from Savoy.

We can make this adjustment using a function available in the *Olm* carbonate water chemistry package for Python. First we have to install it (if you haven't already done so). We can do this using the command below.

One of the critical considerations in analyzing conductivity data, is correction to specific conductance at 25 degrees Celcius. Since temperature also influences raw conductivity, we need to account for this effect by adjusting all measurements to equivalent SpC @ 25 C.

In [2]:
# The function we use for converting to SpC @ 25 C
# is called condTo25. We import it like this.
from olm.general import condTo25

## Long-term SpC records

In [ ]:
# There are long-term SpC records from Langle
# and Copperhead Springs in 'data/Langle.csv' and
# 'data/Copperhead.csv' 

# Read in these data and plot up the time series of
# SpC at Langle and Copperhead. You can use
# these longer records to get a bigger context for the
# SpC data you collected yesterday.

